# Initial Data Processing for IOP2
## Merging sweeps
* author: [Hamid Ali Syed](https://twitter.com/hamidrixvi)
* date: Dec 23, 2022

The program is designed to process data from multiple mobile radars. The data is stored in a set of directories that are organized according to the name of the radar and the date of the data. The program reads the data from these directories and performs several operations on it, including organizing the data into separate directories based on the elevation angle of the radar sweep, and running a tool called `RadxConvert` to process the data.

The program begins by importing several packages that are used throughout the code. These include the `warnings`, `numpy`, `glob`, `os`, `sys`, and `shutil` packages. The warnings package is used to ignore any warnings that may appear during the execution of the code, while the other packages are used for various operations such as searching for files, creating directories, and copying files.

Next, the program defines a function called `process_data` that takes a list of data directories as input and performs the following operations:

1. Iterates through each directory in the list and uses the `glob` package to search for files with the pattern `"*SUR.nc"` in the specified directory. It prints the number of files found for each directory in the list.

2. For each file found, it extracts a numerical value from the file name and stores it as the variable `el`. It then creates a new directory with a name based on the file name, and copies the file into that directory.

3. The function returns a list of the directories it created.

The program then defines the base directory as a string, and the list of data directories as a list of strings. It calls the `process_data` function with the `data_dir` list as input, and stores the output in the `curr_dir_list` variable.

The program then defines a command to modify the user's environment, which is to be a command to load a specific module using the module tool.

Finally, the program iterates through the elements in the `curr_dir_list` and executes the following command for each element:

In [1]:
import warnings
import numpy as np
import glob
import os, sys
import shutil
import subprocess
# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
# Define function to process data
def process_data(data_dir):
    curr_dir_list = []
    for path in data_dir:
        # Find files matching pattern
        files = sorted(glob.glob(os.path.join(basedir, path, "*SUR.nc")))
        print(f"{path}: {len(files)}")
        # Set initial value for el
        el = 99999999
        for f in files:
            # Extract numerical value from file name
            new_el = float(f.split("_")[-2].replace("el", ""))
            if new_el < el:
                # Store new dir name
                curr_dir = f.split(".")[1]
                curr_dir_list.append(os.path.join(basedir, path, curr_dir))
                # Create directory
                os.makedirs(os.path.join(basedir, path, curr_dir), exist_ok=True)
            # Store new el
            el = new_el
            # Copy file to new directory
            shutil.copy(f"{f}", f"{basedir}{os.sep}{path}{os.sep}{curr_dir}{os.sep}{f.split(os.sep)[-1]}")
    return curr_dir_list

In [5]:
# Set base directory
basedir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP2/"
# Set list of data directories
data_dir = ['COW1', "DOW7", "DOW8"]

In [6]:
# Process data and store list of directories
curr_dir_list = process_data(data_dir)

COW1: 1960
DOW7: 1988
DOW8: 1841


This command uses the `RadxConvert` tool to process the files in each directory specified by `new_files`, and outputs the result to the directory specified by `new_path`. The `-ag_all` flag tells `RadxConvert` to aggregate the data in all fields of the input files, and the `-f` flag specifies the input file or directory. The `-outdir` flag specifies the output directory.

The `mod_cmd` variable that is concatenated to the beginning of the command appears to be a command to modify the user's environment, potentially by loading a specific module. This may be necessary in order to make the `RadxConvert` tool available to the user.

The command is then executed using the `subprocess.check_output` function, which runs the command and captures the output. If an error occurs during the execution of the command, it will raise a `CalledProcessError` exception, which the code handles by printing the error output.

In [ ]:
# Set command to modify environment
mod_cmd = "module use /depot/dawson29/data/Projects/PERiLS/modulefiles/ && module load lrose"

# Iterate through directories and process data
for new_files in curr_dir_list:
    # Set output directory
    new_path = os.path.join(new_files.split("202203")[0], "cmerged")
    # Build command to run RadxConvert
    cmd = f"{mod_cmd} && RadxConvert -ag_all -f {new_files}/* -outdir {new_path}"
    # Run command using subprocess
    try:
        output = subprocess.check_output(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        # Handle error
        print(e.output)